In [1]:
import time
import numpy as np
from time import sleep
from emd.sift import sift
import socketio
import tensorflow as tf
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

COM_PORT = 'COM3'    # 指定通訊埠名稱
NO_ARDUINO = True
REAL_DATA = True
BAUD_RATE = 115200    # 設定傳輸速率
CHANNEL_NUMBER = 3
SAMPLE_TIME = 1 / 500
WINDOW_SIZE = 100
THRESHOLD = 0.5
NUM_IMF = 6
RECEIVE_CHANNEL = 'ArduinoSignal'
EMIT_CHANNEL = 'InferenceResults'

ACTIONS = ['undefined action', 'up', 'down', 'left', 'right', 'quick touch']
MAXCHARLEN = max([len(x) for x in ACTIONS])

model_path = './model/LickingPark'
server_url = 'http://localhost:3000'

In [2]:
class inference():
    def __init__(self, url, modelPath) -> None:
        self.__model = tf.keras.models.load_model(modelPath)
        self.__sio = socketio.Client()
        self.__sio.connect(url)
        try:
            assert isinstance(self._sio, socketio.Client)
            print("Connected to Socket server.")
        except:
            raise Exception("Maybe server is not online.")
        self.__container = None
        
    def __revieveSignal(self, raw):
        try: # received CHANNEL_NUMBER * WINDOW_SIZE size of data
            self.__container = np.array(raw.rstrip(',').split(",")).astype(np.float32).reshape(CHANNEL_NUMBER, WINDOW_SIZE).T
        except: # received CHANNEL_NUMBER size of data
            data = np.array(raw.split(",")).astype(np.float32)
            if isinstance(self.__container, np.ndarray):
                if self.__container.shape[0] < WINDOW_SIZE:
                    self.__container = np.concatenate([self.__container, data[np.newaxis,:]], axis=0)
                else:
                    self.__container = np.concatenate([self.__container[1:], data[np.newaxis,:]], axis=0)
            else:
                self.__container = data[np.newaxis,:]
    
    def run(self):
        print("Starting inference.")
        self.__sio.on(RECEIVE_CHANNEL, self.__revieveSignal)
        while True:
            try:
                data = self.__container.copy()
                assert data.shape[1] == WINDOW_SIZE
                # timer = time.time()
                inputData = self.__emdSignal(data)
                # emdTime = time.time() - timer
                # timer = time.time()
                res = self.__model.predict(inputData,
                                          verbose = False)
                # print(res)
                # print("Spend {:.3f}s handling signal, {:.3f}s predicting, the result is {}.".format(emdTime, time.time() - timer, res))
                print(ACTIONS[np.argmax(res)].ljust(MAXCHARLEN), end='\r')
                self.__sio.emit(EMIT_CHANNEL, ACTIONS[np.argmax(res)])
            except KeyboardInterrupt:
                self.__sio.disconnect()
                break 
            except:
                continue #it's a None
                   
        print("Inference finished.")
        
    def __emdSignal(self, sig):
        channel = sig.shape[-1]
        ret = None
    
        for c in range(channel):
            raw = sig[0, :, c]
            imf = sift(raw, max_imfs=NUM_IMF, imf_opts={'sd_thresh': 0.1})
            
            if imf.shape[-1] < NUM_IMF:
                compensate = np.zeros((WINDOW_SIZE, NUM_IMF - imf.shape[-1]))
                imf = np.concatenate([imf, compensate], axis = 1)
            
            if not type(ret) == np.ndarray: 
                ret = imf
            else: 
                ret = np.concatenate([ret, imf], axis = 1)
                     
        return ret[np.newaxis, :]

In [4]:
emdCNN = inference(server_url, model_path)
emdCNN.run()

Connected to Socket server.
Starting inference.
